In [12]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [13]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv(shape = [4, 4])

In [14]:
# Taken from Policy Evaluation Exercise!

def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        delta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # For each action, look at the possible next states...
                for  prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # Stop evaluating once our value function change is below a threshold
        if delta < theta:
            break
    return np.array(V)

In [15]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA

    while True:

        # Evaluation current policy
        v = policy_eval_fn(policy, env, discount_factor)
        
        # will be set to false if any changes is made for the policy
        policy_stable = True
        
        # for each state...
        for s in range(env.nS):
        
            # best action under current policy
            current_action = np.argmax(policy[s])
            
            # find the best action by one-step look ahead
            action_values = np.zeros(env.nA)
            for a, _ in enumerate(policy[s]):
                prob, next_state, reward, done = env.P[s][a][0]
                action_values[a] = prob * (reward + discount_factor * v[next_state])
            
            # greedily select best policy 
            best_action = np.argmax(action_values)
            
            if current_action != best_action:
                policy_stable = False
                
            # update policy 
            policy[s, best_action] = 1.0
            
                
        if policy_stable:
            return policy, v
    
    

In [16]:
# time 
import time
start = time.time()
policy, v = policy_improvement(env)
end = time.time()
duration = end - start
print('Policy iteration time: {}'.format(duration))
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")



/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:31: RuntimeWarning: overflow encountered in double_scalars
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in double_scalars


Policy iteration time: 0.2701289653778076
Policy Probability Distribution:
[[ 1.    0.25  0.25  0.25]
 [ 0.25  0.25  0.25  1.  ]
 [ 1.    0.25  0.25  1.  ]
 [ 1.    0.25  0.25  1.  ]
 [ 1.    0.25  0.25  0.25]
 [ 1.    0.25  0.25  0.25]
 [ 1.    0.25  0.25  1.  ]
 [ 1.    0.25  1.    0.25]
 [ 1.    0.25  0.25  0.25]
 [ 1.    0.25  0.25  0.25]
 [ 1.    1.    0.25  0.25]
 [ 0.25  0.25  1.    0.25]
 [ 1.    0.25  0.25  0.25]
 [ 1.    1.    0.25  0.25]
 [ 0.25  1.    0.25  0.25]
 [ 1.    0.25  0.25  0.25]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 0 0]
 [0 0 0 0]
 [0 0 0 2]
 [0 0 1 0]]

Value Function:
[  0. -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
   0.]

Reshaped Grid Value Function:
[[  0. -inf -inf -inf]
 [-inf -inf -inf -inf]
 [-inf -inf -inf -inf]
 [-inf -inf -inf   0.]]



In [11]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)